In [ ]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime
import pandas as pd
import numpy as np
import math
from scipy.optimize import brentq
import xarray as xr

from sampling import (NestedSampling, Uniform,
                      Callback, Normal, Constant,
                      SamplingException)

from filterpy.kalman import UnscentedKalmanFilter as UKF
from filterpy.kalman import KalmanFilter as KF
from filterpy.kalman import MerweScaledSigmaPoints
from filterpy.common import Q_continuous_white_noise

import progressbar

from clemb.forward_model import forward_model, fullness, esol
from clemb import clemb, LakeDataFITS, WindDataCSV

## Table of contents
---------------------------
1. [Inversion](#Inversion)
2. [Forecasting](#Forecasting)

## Inversion

In [ ]:
startdate = '2018-05-01'
tend = '2018-09-25'
c = clemb.Clemb(LakeDataFITS(), WindDataCSV(), start=startdate, end=tend)

In [ ]:
c.drmg = True
rsb = c.run_backward()

In [ ]:
rs = c.run_forward(nsamples=5000,)

In [ ]:
days = mdates.DayLocator()  # every day
months = mdates.MonthLocator()
monthFmt = mdates.DateFormatter('%Y-%m-%d')
dayFmt = mdates.DateFormatter('%d')

datet = rs['dates']
data = c._df
t_data = data.index
nsteps = datet.size
nresample = 500

mpl.rcParams['figure.subplot.hspace'] = 0.5
fig, axs = plt.subplots(nrows=5, ncols=2, figsize=(14, 10))

axs[0,0].plot(t_data, np.ones(t_data.size)*4.5, ls='--')
axs[0,0].set_title('Wind speed [m/s]')

axs[0,1].plot(t_data, data['X'], ls='--')

model_X = rs['model'].sel(obs='X')
axs[0,1].fill_between(t_data, data['X']-3*data['X_err'],
                      data['X']+3*data['X_err'], alpha=0.5)
axs[0,1].plot(datet, model_X.mean(axis=1), 'k-')
axs[0,1].plot(datet, model_X.mean(axis=1)+3*model_X.std(axis=1), 'k--')
axs[0,1].plot(datet, model_X.mean(axis=1)-3*model_X.std(axis=1), 'k--')
axs[0,1].set_title('Mg++ amount [kt]')

axs[1,0].plot(t_data, data['T'], ls='--')
axs[1,0].fill_between(t_data, data['T']-3*data['T_err'],
                      data['T']+3*data['T_err'], alpha=0.5)

model_T = rs['model'].sel(obs='T')
axs[1,0].plot(datet, model_T.mean(axis=1), 'k-')
axs[1,0].plot(datet, model_T.mean(axis=1)+3*model_T.std(axis=1), 'k--')
axs[1,0].plot(datet, model_T.mean(axis=1)-3*model_T.std(axis=1), 'k--')
axs[1,0].set_title('Lake temperature [$^{\circ}C$]')

axs[1,1].plot(t_data, data['M'], ls='--')
axs[1,1].fill_between(t_data, data['M']-3*data['M_err'],
                      data['M']+3*data['M_err'], alpha=0.5)
model_M = rs['model'].sel(obs='M')
axs[1,1].plot(datet, model_M.mean(axis=1), 'k-')
axs[1,1].plot(datet, model_M.mean(axis=1)+3*model_M.std(axis=1), 'k--')
axs[1,1].plot(datet, model_M.mean(axis=1)-3*model_M.std(axis=1), 'k--')
axs[1,1].set_title('Lake mass [kt]')

for k in range(nsteps):
    axs[2,0].scatter([datet[k].data]*nresample, rs['q_in'][k], s=2, c=rs['lh'][k],
                     cmap=plt.cm.get_cmap('RdBu_r'), alpha=0.3)
exp_q_in = rs['exp'].loc[:,'q_in']
var_q_in = rs['var'].loc[:,'q_in']
axs[2,0].plot(datet, exp_q_in, 'k')
axs[2,0].plot(datet, exp_q_in - 3*np.sqrt(var_q_in), 'k--')
axs[2,0].plot(datet, exp_q_in + 3*np.sqrt(var_q_in), 'k--')
axs[2,0].plot(rsb['dates'], rsb['pwr'], 'b-')
axs[2,0].set_title('Heat input rate [MW]')

for k in range(nsteps):
    axs[2,1].scatter([datet[k].data]*nresample, rs['h'][k], s=2, c=rs['lh'][k],
                cmap=plt.cm.get_cmap('RdBu_r'), alpha=0.3)
exp_h = rs['exp'].loc[:,'h']
axs[2,1].plot(datet, exp_h, 'k')
#axs[2,1].plot(np.arange(nsteps), exp[:,3] - 3*np.sqrt(var[:,3]), 'k--')
#axs[2,1].plot(np.arange(nsteps), exp[:,3] + 3*np.sqrt(var[:,3]), 'k--')
#axs[2,1].plot(np.arange(nsteps+1), np.ones(ndf.index.size)*6.0, ls='--')
axs[2,1].set_title('Enthalpy [TJ/kt]')

for k in range(nsteps):
    axs[3,0].scatter([datet[k].data]*nresample, rs['m_in'][k], s=2, c=rs['lh'][k],
                cmap=plt.cm.get_cmap('RdBu_r'), alpha=0.3)
exp_m_in = rs['exp'].loc[:,'m_in']
var_m_in = rs['var'].loc[:,'m_in']
axs[3,0].plot(datet, exp_m_in, 'k')
axs[3,0].plot(datet, exp_m_in - 3*np.sqrt(var_m_in), 'k--')
axs[3,0].plot(datet, exp_m_in + 3*np.sqrt(var_m_in), 'k--')
axs[3,0].plot(rsb['dates'], rsb['fmelt'], 'b-')
axs[3,0].set_ylim(-50, 50)
axs[3,0].set_title('Inflow [kt/day]')

for k in range(nsteps):
    axs[3,1].scatter([datet[k].data]*nresample, rs['m_out'][k], s=2, c=rs['lh'][k],
                cmap=plt.cm.get_cmap('RdBu_r'), alpha=0.3)
exp_m_out = rs['exp'].loc[:,'m_out']
var_m_out = rs['var'].loc[:,'m_out']
axs[3,1].plot(datet, exp_m_out, 'k')
axs[3,1].plot(datet, exp_m_out - 3*np.sqrt(var_m_out), 'k--')
axs[3,1].plot(datet, exp_m_out + 3*np.sqrt(var_m_out), 'k--')
axs[3,1].plot(rsb['dates'], rsb['inf'], 'b-')
axs[3,1].set_ylim(-50, 50)
axs[3,1].set_title('Outflow [kt/day]')

for k in range(nsteps):
    axs[4,0].scatter([datet[k].data]*nresample, rs['steam'][k], s=2, c=rs['lh'][k],
                     cmap=plt.cm.get_cmap('RdBu_r'), alpha=0.3)
axs[4,0].plot(rsb['dates'], rsb['steam'], 'b-')
axs[4,0].set_title('Steam input [kt/day]')
    
for k in range(nsteps):
    axs[4,1].scatter([datet[k].data]*nresample, rs['mevap'][k], s=2, c=rs['lh'][k],
                     cmap=plt.cm.get_cmap('RdBu_r'), alpha=0.3)
axs[4,1].plot(rsb['dates'], rsb['evfl'], 'b-')
_ = axs[4,1].set_title('Evaporation mass loss [kt/day]')

for row in range(5):
    for col in range(2):
        axs[row,col].xaxis.set_major_locator(months)
        axs[row,col].xaxis.set_major_formatter(monthFmt)
        axs[row,col].xaxis.set_minor_locator(days)
        axs[row,col].set_xlim(datet[0].data, datet[-1].data)
fig.savefig('inference_result.png')

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(12, 10))
axs[0,0].hist(np.ma.masked_invalid(rs['q_in'][-1]).compressed(), bins=20)
ylim = axs[0,0].get_ylim()
exp_q_in = rs['exp'].loc[:,'q_in'][-1]
var_q_in = rs['var'].loc[:,'q_in'][-1]
axs[0,0].vlines(exp_q_in, *ylim)
axs[0,0].vlines(exp_q_in - np.sqrt(var_q_in), *ylim, linestyle='--')
axs[0,0].vlines(exp_q_in + np.sqrt(var_q_in), *ylim, linestyle='--')
axs[0,0].set_ylim(*ylim)
axs[0,0].set_xlabel('Heat input rate [MW]')

axs[0,1].hist(np.ma.masked_invalid(rs['steam'][-1]).compressed(), bins=20)
ylim = axs[0,1].get_ylim()
mn_stm = rs['steam'][-1].mean()
std_stm = rs['steam'][-1].std()
axs[0,1].vlines(mn_stm, *ylim)
axs[0,1].vlines(mn_stm - std_stm, *ylim, linestyle='--')
axs[0,1].vlines(mn_stm + std_stm, *ylim, linestyle='--')
axs[0,1].set_ylim(*ylim)
axs[0,1].set_xlabel('Steam input rate [kt/day]')

axs[1,0].hist(np.ma.masked_invalid(rs['m_in'][-1]).compressed(), bins=20)
exp_m_in = rs['exp'].loc[:,'m_in'][-1]
var_m_in = rs['var'].loc[:,'m_in'][-1]
axs[1,0].vlines(exp_m_in, *ylim)
axs[1,0].vlines(exp_m_in - np.sqrt(var_m_in), *ylim, linestyle='--')
axs[1,0].vlines(exp_m_in + np.sqrt(var_m_in), *ylim, linestyle='--')
axs[1,0].set_ylim(*ylim)
axs[1,0].set_xlabel('Water inflow rate [kt/day]')

axs[1,1].hist(np.ma.masked_invalid(rs['m_out'][-1]).compressed(), bins=20)
exp_m_out = rs['exp'].loc[:,'m_out'][-1]
var_m_out = rs['var'].loc[:,'m_out'][-1]
axs[1,1].vlines(exp_m_out, *ylim)
axs[1,1].vlines(exp_m_out - np.sqrt(var_m_out), *ylim, linestyle='--')
axs[1,1].vlines(exp_m_out + np.sqrt(var_m_out), *ylim, linestyle='--')
axs[1,1].set_ylim(*ylim)
_ = axs[1,1].set_xlabel('Water outflow rate [kt/day]')

## Forecasting

In [ ]:
def fullness_inv(v):
    """
    Compute lake level from lake volume.
    """
    def f(h, v):
        return (4.747475*np.power(h, 3)-34533.8*np.power(h, 2) + 83773360.*h-67772125000.)/1000. - v
    return brentq(f, 2400, 2600, args=v)

def f_x(x, dt, dQin, month, time, H, ws):
    """
    Forward model lake temperature
    """
    Q_in = x[0]
    M_in = x[1]
    M_out = x[2]
    T = x[3]
    M = x[4]
    X = x[5]
    a = x[6]
    v = x[7]
    y = np.array([T, M, X])
    solar = esol(dt, a, month)
    y_next, st, me = forward_model(y, dt, a, v, Q_in * 0.0864, 
                                   M_in, M_out, solar, H, ws)
    p0 = 1.003 - 0.00033 * T
    p1 = 1.003 - 0.00033 * y_next[0]
    v0 = M/p0
    v1 = y_next[1]/p1
    h0 = fullness_inv(v0)
    h1 = fullness_inv(v1)
    dh = h1 - h0
    a_next = (v1 - v0)*1e3/(h1 - h0)
    q_in_next = Q_in + dQin * dt
    return np.array([q_in_next, M_in, M_out, 
                     y_next[0], y_next[1], y_next[2],
                     a_next, v1])

def h_x(x):
    """
    Measurement function
    """
    return [x[0]]

dt = 1.
points = MerweScaledSigmaPoints(n=8, alpha=.1, beta=2., kappa=0.)
kf = UKF(dim_x=8, dim_z=1, dt=dt, fx=f_x, hx=h_x, points=points)
x = np.r_[rs['exp'][-1,0:3].data, rs['exp'][-1,4:].data]
P = np.eye(8)*np.r_[rs['var'][-1,0:3].data,rs['var'][-1,4:].data]
kf.x = x
kf.Q = np.eye(8)*1e-7
kf.P = P
kf.R = 50.**2
nperiods=10
datetime_new = pd.date_range(start=datet[-1].data, periods=nperiods, freq='D')
dQin = rs['exp'][-1,0].data - rs['exp'][-2,0].data
means = np.zeros((nperiods, 8))
covariances = np.zeros((nperiods, 8, 8))
for i in range(nperiods):
    kf.predict(dQin=dQin, time=i*dt,
               month=datetime_new.month[i],
               H=rs['exp'][-1,3].data,
               ws=4.5)
    means[i,:] = kf.x
    covariances[i, :, :] = kf.P

In [ ]:
T_pred = np.r_[data['T'], means[:,3]]
dates = np.r_[t_data, datetime_new+1]
nvar = np.r_[data['T_err'], np.sqrt(covariances[:, 3, 3])]
plt.figure(figsize=(12,6))
plt.plot(dates, T_pred)
plt.fill_between(dates, T_pred -2*nvar, T_pred + 2*nvar, alpha=0.5)
_ = plt.ylabel(r'Temperature [$^\circ$C]')
plt.savefig('RCL_T_prediction.png')